# Asymmetry tests

Hypothesis: the absolute value in the definition of asymmetry is **problematic**. CAS asymmetry is defined as:

$$ A = \frac{\sum |f - f^{180}|}{\sum |f| } - A_{bg}$$

where 

$$ A_{bg} = \frac{n_{obj}}{n_{sky}} \frac{\sum |f_{sky} - f_{sky}^{180}|}{\sum |f|}$$

so to expand, asymmetry is calculated as

$$ A = \frac{\sum |f - f^{180}| - \frac{n_{img}}{n_{sky}} \sum |f_{sky} - f_{sky}^{180}|}{\sum |f| } $$

This definition has a dependence on the noise, and therefore simply subtracting the background term doesn't account for all the noise in the image. The absolute value adds a correlation between the source asymmetry and the background term, even for a Gaussian background. If we used squared residuals rather than absolute residuals, this would not be a problem.

Changes to asymmetry that we test here:

1. Replace the residual calculation from absolute value to *squared*, which has a separable noise contribution
2. Add a correction term in the denominator that accounts for the contribution of BG flux to total
3. Change the initial guess for asymmetry center from center of flux to center of flux$^2$ for better performance

#### Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import galsim

from asymmetry import get_asymmetry

## Create SDSS-like image

Utility function to make an SDSS-like image using [**GalSim**](https://github.com/GalSim-developers/GalSim) and `galaxy_generator.py` functions.

Arguments:

* `mag`: source magnitude
* `sky_mag`: sky surface brightness in mag/arcsec$^2$
* `r_eff`: 

**<font color='#aa33ff'>Questions:</font>**

1. How are telescope and transmission parameters defined?

In [ ]:
def make_sdss_galaxy(mag, sky_mag, re):
    
    filt = 'r'
    bandpass_file = "passband_sdss_" + Filter
    bandpass = galsim.Bandpass(bandpass_file, wave_type = u.angstrom)

    telescope_params = {'g':4.8, 't_exp':53.91, 'D':2.5}
    transmission_params = {'eff_wav':616.5, 'del_wav':137}

mag = 13
sky_mag = 22 ##mag/arcsec/arcsec
re = 10 #arcsec

galaxy = gen_galaxy(mag, re=re, n=1, q=1, beta=0, psf_sig=2, telescope_params=telescope_params, 
	transmission_params=transmission_params, bandpass=bandpass)

centre_ra = 150
centre_dec = 2.3
pixel_scale = 0.4
fov = re*12/3600 #deg


image, wcs = gen_image(galaxy, centre_ra, centre_dec, pixel_scale, fov, fov)

rp = petrosian_sersic(fov, re, 1)/pixel_scale  ##in pixels


image = add_asymmetry(image, rp, 12, 2.0,  mag, telescope_params, transmission_params, bandpass)

# after Poisson noise is added, change to ADU by dividing by gain
image = sky_noise(image, sky_mag, pixel_scale, telescope_params, transmission_params, bandpass)/telescope_params['g']



